Ejercico 1

In [8]:
import pandas as pd
df = pd.read_csv('/content/crime_data.csv')

df.head()

,State,VR,MR,M,W,H,P,S
0,AK,761,9.0,41.8,75.2,86.6,9.1,14.3
1,AL,780,11.6,67.4,73.5,66.9,17.4,11.5
2,AR,593,10.2,44.7,82.9,66.3,20.0,10.7
3,AZ,715,8.6,84.7,88.6,78.7,15.4,12.1
4,CA,1078,13.1,96.7,79.3,76.2,18.2,12.5


In [9]:
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

X = df[['M', 'W', 'H', 'P', 'S']].values
y = df['MR'].values

regr = MLPRegressor(hidden_layer_sizes=(20, 20), max_iter=10000, random_state=42)
regr.fit(X, y)
y_pred = regr.predict(X)

print('MSE en el conjunto completo: ', mean_squared_error(y, y_pred))

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

mse = 0

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index, :], X[test_index, :]
    y_train, y_test = y[train_index], y[test_index]

    regr_cv = MLPRegressor(hidden_layer_sizes=(20, 20), max_iter=10000, random_state=42)
    regr_cv.fit(X_train, y_train)

    y_pred = regr_cv.predict(X_test)

    mse_i = mean_squared_error(y_test, y_pred)
    print('MSE en pliegue actual: ', mse_i)
    mse += mse_i

mse = mse / n_splits
print('MSE promedio con validación cruzada: ', mse)

MSE en el conjunto completo:  2.58198658990913
MSE en pliegue actual:  7.107750264511864
MSE en pliegue actual:  6.150984626593375
MSE en pliegue actual:  3.5177821154402813
MSE en pliegue actual:  3.9245598076292167
MSE en pliegue actual:  142.71103108298811
MSE promedio con validación cruzada:  32.68242157943257


1. ¿Consideras que el modelo perceptrón multicapa es efectivo para modelar los datos del problema? ¿Por qué?
No, el modelo de perceptrón multicapa no parece ser efectivo para modelar los datos. Aunque el MSE en el conjunto completo es relativamente bajo (2.58), el MSE promedio con validación cruzada (32.68) es mucho mayor, con una alta variabilidad entre los pliegues (particularmente el último pliegue, que tiene un MSE de 142.71). Esto indica que el modelo puede estar sobreajustándose a los datos de entrenamiento

2. ¿Qué modelo es mejor para los datos de criminalidad, el lineal o el perceptrón multicapa? ¿Por qué?
Es probable que un modelo lineal sea mejor para los datos de criminalidad, dado que el modelo de perceptrón multicapa muestra signos de sobreajuste y gran variabilidad en su desempeño durante la validación cruzada.

Ejercicio 2

2.1

In [4]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

data = np.loadtxt('/content/M_3.txt')
y = data[:, 0]
X = data[:, 2:]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Validación cruzada
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True)
cv_y_test = []
cv_y_pred = []

for train_index, test_index in kf.split(X_scaled, y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = MLPClassifier(hidden_layer_sizes=(20, 20, 20, 20, 20), max_iter=10000)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    cv_y_test.append(y_test)
    cv_y_pred.append(y_pred)

print(classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred)))


              precision    recall  f1-score   support

         1.0       0.99      0.98      0.98        90
         2.0       0.92      0.93      0.93        90
         3.0       0.94      0.92      0.93        90
         4.0       1.00      0.97      0.98        90
         5.0       0.98      0.99      0.98        90
         6.0       0.89      0.91      0.90        90
         7.0       0.96      0.98      0.97        90

    accuracy                           0.95       630
   macro avg       0.95      0.95      0.95       630
weighted avg       0.95      0.95      0.95       630



2.2

In [6]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

y = data[:, 0]
X = data[:, 2:]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

num_layers = np.arange(1, 6)  # Probar de 1 a 5 capas
num_neurons = np.arange(10, 110, 20)  # Probar de 10 a 100 neuronas con saltos de 20
layers = []

for l in num_layers:
    for n in num_neurons:
        layers.append(tuple(l * [n]))

clf = GridSearchCV(MLPClassifier(max_iter=10000), {'hidden_layer_sizes': layers}, cv=5)
clf.fit(X_scaled, y)

print("Mejor estimador:", clf.best_estimator_)

y_pred = cross_val_predict(clf.best_estimator_, X_scaled, y, cv=5)
print(classification_report(y, y_pred))

Mejor estimador: MLPClassifier(hidden_layer_sizes=(70, 70, 70, 70, 70), max_iter=10000)
              precision    recall  f1-score   support

         1.0       0.97      0.97      0.97        90
         2.0       0.94      0.97      0.95        90
         3.0       0.91      0.92      0.92        90
         4.0       0.99      0.98      0.98        90
         5.0       0.97      0.97      0.97        90
         6.0       0.93      0.89      0.91        90
         7.0       0.93      0.94      0.94        90

    accuracy                           0.95       630
   macro avg       0.95      0.95      0.95       630
weighted avg       0.95      0.95      0.95       630



2.3

In [7]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

y = data[:, 0]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

num_layers = np.arange(1, 6)  # Probar de 1 a 5 capas
num_neurons = np.arange(10, 110, 20)  # Probar de 10 a 100 neuronas con saltos de 20
layers = []

for l in num_layers:
    for n in num_neurons:
        layers.append(tuple(l * [n]))

clf_grid = GridSearchCV(MLPClassifier(max_iter=10000), {'hidden_layer_sizes': layers}, cv=5)
clf_grid.fit(X_scaled, y)

best_clf = clf_grid.best_estimator_
print("Hiperparámetros óptimos encontrados:", best_clf)

# Ajustar el modelo con todos los datos utilizando los hiperparámetros óptimos
best_clf.fit(X_scaled, y)

# Realizar predicciones con el modelo ajustado
y_pred = best_clf.predict(X_scaled)

print("Resultados del modelo ajustado con todos los datos:")
print(classification_report(y, y_pred))

Hiperparámetros óptimos encontrados: MLPClassifier(hidden_layer_sizes=(10,), max_iter=10000)
Resultados del modelo ajustado con todos los datos:
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00        90
         2.0       1.00      1.00      1.00        90
         3.0       1.00      0.97      0.98        90
         4.0       1.00      1.00      1.00        90
         5.0       1.00      1.00      1.00        90
         6.0       1.00      0.97      0.98        90
         7.0       0.94      1.00      0.97        90

    accuracy                           0.99       630
   macro avg       0.99      0.99      0.99       630
weighted avg       0.99      0.99      0.99       630



1. ¿Observas alguna mejora importante al optimizar el tamaño de la red? ¿Es el resultado que esperabas? Argumenta tu respuesta.

Sí, se observa una mejora al optimizar el tamaño de la red. El modelo con los hiperparámetros óptimos (MLPClassifier(hidden_layer_sizes=(10,))) alcanza una precisión general del 99%, lo que representa un incremento en comparación con los resultados iniciales que tenían una precisión del 95%. Esta mejora es significativa, especialmente considerando que el modelo más simple (con solo 10 neuronas en una capa) logra un rendimiento superior al modelo más complejo (5 capas de 70 neuronas cada una).


2. ¿Qué inconvenientes hay al encontrar el tamaño óptimo de la red? ¿Por qué?

Pues aunque encontrar el tamaño óptimo de la red puede mejorar el rendimidento, también puede ser un proceso costoso computacionalmente hablando y propenso al sobreajuste, lo que requiere una cuidadosa evaluación y validación para asegurar que los beneficios compensen los inconvenientes.









Ejercicio 3

3.1

In [2]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

data = np.loadtxt('/content/P1_5.txt')
y = data[:, 0]
X = data[:, 2:]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

clf = SGDClassifier(loss='perceptron', max_iter=1000, tol=1e-3, random_state=42)
y_pred = cross_val_predict(clf, X_scaled, y, cv=5)

print("Resultados del modelo de perceptrón entrenado con descenso de gradiente estocástico:")
print(classification_report(y, y_pred))

Resultados del modelo de perceptrón entrenado con descenso de gradiente estocástico:
              precision    recall  f1-score   support

         1.0       0.63      0.57      0.60       281
         2.0       0.93      0.94      0.94      1689

    accuracy                           0.89      1970
   macro avg       0.78      0.76      0.77      1970
weighted avg       0.89      0.89      0.89      1970



3.2

In [3]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

y = data[:, 0]
X = data[:, 2:]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Descenso de Gradiente por Lotes (Batch Gradient Descent)
# Ajustar el tamaño del lote al número total de muestras
batch_size = X_scaled.shape[0]

clf_batch = MLPClassifier(hidden_layer_sizes=(1,), max_iter=1000, solver='sgd', batch_size=batch_size, random_state=42)
y_pred_batch = cross_val_predict(clf_batch, X_scaled, y, cv=5)

print("Resultados del modelo de perceptrón entrenado con descenso de gradiente por lotes:")
print(classification_report(y, y_pred_batch))

# Descenso de Gradiente de Mini-Lote (Mini-Batch Gradient Descent)
# Ajustar un tamaño de mini-lote, por ejemplo, 32
mini_batch_size = 32

clf_minibatch = MLPClassifier(hidden_layer_sizes=(1,), max_iter=1000, solver='sgd', batch_size=mini_batch_size, random_state=42)
y_pred_minibatch = cross_val_predict(clf_minibatch, X_scaled, y, cv=5)

print("Resultados del modelo de perceptrón entrenado con descenso de gradiente de mini-lote:")
print(classification_report(y, y_pred_minibatch))

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger tha

Resultados del modelo de perceptrón entrenado con descenso de gradiente por lotes:
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00       281
         2.0       0.86      1.00      0.92      1689

    accuracy                           0.86      1970
   macro avg       0.43      0.50      0.46      1970
weighted avg       0.74      0.86      0.79      1970



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Resultados del modelo de perceptrón entrenado con descenso de gradiente de mini-lote:
              precision    recall  f1-score   support

         1.0       0.67      0.68      0.68       281
         2.0       0.95      0.94      0.95      1689

    accuracy                           0.91      1970
   macro avg       0.81      0.81      0.81      1970
weighted avg       0.91      0.91      0.91      1970



3.3

In [6]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler

y = data[:, 0]
X = data[:, 2:]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_categorical = to_categorical(y - 1)

kf = StratifiedKFold(n_splits=5, shuffle=True)
cv_y_test = []
cv_y_pred = []

# Configurar la detención temprana
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

for train_index, test_index in kf.split(X_scaled, y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    y_train_categorical = to_categorical(y_train - 1)
    y_test_categorical = to_categorical(y_test - 1)

    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(2, activation='softmax'))  # Dos clases, usar softmax

    # Compilar el modelo
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrenar el modelo con EarlyStopping
    model.fit(X_train, y_train_categorical, validation_data=(X_test, y_test_categorical),
              epochs=50, batch_size=5, verbose=1, callbacks=[early_stopping])

    # Evaluación en fase de prueba
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    cv_y_test.append(y_test)
    cv_y_pred.append(y_pred)

print("Resultados del modelo de perceptrón multicapa con validación cruzada:")
print(classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred)))


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


316/316 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6692 - loss: 0.6048 - val_accuracy: 0.8858 - val_loss: 0.4019
Epoch 2/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8836 - loss: 0.2799 - val_accuracy: 0.8934 - val_loss: 0.4011
Epoch 3/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9378 - loss: 0.1621 - val_accuracy: 0.9010 - val_loss: 0.4409
Epoch 4/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9553 - loss: 0.1307 - val_accuracy: 0.8985 - val_loss: 0.4942
Epoch 5/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9628 - loss: 0.1090 - val_accuracy: 0.8909 - val_loss: 0.5438
Epoch 6/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9747 - loss: 0.0852 - val_accuracy: 0.8832 - val_loss: 0.6206
Epoch 7/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9852 - loss: 0.0572 - val_accuracy: 0.8858 - val_loss: 0.6964
Epoch 8/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9871 - loss: 0.0568 - val_accuracy: 0.8909 - val_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

1. ¿El modelo de una neurona es suficiente para modelar el conjunto de datos de este problema?
No, el modelo de una neurona no es suficiente para modelar el conjunto de datos de este problema. Aunque el modelo entrenado con descenso de gradiente estocástico y el de mini-lote muestran una precisión general relativamente alta (89% y 91%), esto se debe al desempeño en la clase mayoritaria (clase 2). El modelo tiene un rendimiento deficiente en la clase minoritaria (clase 1), con un recall de solo 57% en el caso de descenso de gradiente estocástico y de 68% en el caso de mini-lote.

